# GerryFair Tutorial

## Required Data Format

In order to train and audit for bias, we require three dataframes. The first two are the standard `X` and `Y`, which are the samples and their labels respectively. These should be one-hot-encoded. The third required dataframe is the list of protected attributes. This contain the values of the protected attributes for each row in the sample. These will be the attributes that we will audit for bias towards. Please note that we do not promise to protect against bias towards attributes that are not included in this list.

#### Cleaning Data
If your data is not in that format, it needs to be cleaned. We provide a method, `clean_data` in *clean.py*, you can use to clean your data into the accepted format.

The variable `dataset` should hold the file path to the file containing the dataset. The variable `attributes` should hold the file path to the file containing protected attributes. This should simply be one row where a column is 2 if it is the label, 1 if it is protected, and 0 otherwise. You should set `centered` to be `True` if you want the data to be centered.

In [10]:
import gerryfair.gerryfair
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
dataset = "./preprocessed.csv"
attributes = "./protected.csv"
centered = True
X, X_prime, y = gerryfair.clean.clean_dataset(dataset, attributes, centered)

label feature: ['completed']
sensitive features: ['country_cd_US', 'is_female', 'bachelor_obtained', 'white']
there are 4 sensitive features including derivative features


## Using tools to train a model

Now, we can use the `Model` class in order to train a new model. When instatiating the object, you may provide any options that you will want when training the classifier. If you wish to change the options, you may use the `set_options` method. Both are shown below.

In [11]:
C = 15
printflag = True
gamma = .01
fair_model = gerryfair.model.Model(C=C, printflag=printflag, gamma=gamma)
max_iters = 500
fair_model.set_options(max_iters=max_iters)

Now that we are happy with the options, we can use the `train` method to train a classifier using the Fictitious Play algorithm described in [the original paper](https://arxiv.org/abs/1711.05144v3). We will need our three dataframes from earlier. We also return the final errors and fp_difference from training.

I will first split `X` and `y` into a training and a testing set.

In [12]:
# Train Set
X_train = X.iloc[:X.shape[0]-50]
X_prime_train = X_prime.iloc[:X_prime.shape[0]-50]
y_train = y.iloc[:y.shape[0]-50]
# Test Set
X_test = X.iloc[-50:].reset_index(drop=True)
X_prime_test = X_prime.iloc[-50:].reset_index(drop=True)
y_test = y.iloc[-50:].reset_index(drop=True)

# Train the model
[errors, fp_difference] = fair_model.train(X_train, X_prime_train, y_train)

iteration: 1
most accurate classifier accuracy: 0.21851145038167938, most acc-class unfairness: 0.02206516582567819, most acc-class size 0.39885496183206104
ave error: 0.218511, gamma-unfairness: 0.022065, group_size: 0.398855, frac included ppl: 0.666985
iteration: 2
ave error: 0.218511, gamma-unfairness: 0.022065, group_size: 0.398855, frac included ppl: 0.666985
iteration: 3
ave error: 0.352417, gamma-unfairness: 0.015131, group_size: 0.152672, frac included ppl: 1.000000
iteration: 4
ave error: 0.419370, gamma-unfairness: 0.031407, group_size: 0.398855, frac included ppl: 1.000000
iteration: 5
ave error: 0.407061, gamma-unfairness: 0.029832, group_size: 0.398855, frac included ppl: 1.000000
iteration: 6
ave error: 0.387723, gamma-unfairness: 0.021812, group_size: 0.098282, frac included ppl: 1.000000
iteration: 7
ave error: 0.383860, gamma-unfairness: 0.019131, group_size: 0.436069, frac included ppl: 1.000000
iteration: 8
ave error: 0.379175, gamma-unfairness: 0.016018, group_size

We can now use our model to make out of sample predictions. This can be done using the `predict` method of the object.

In [13]:
predictions = fair_model.predict(X_test)

## Using tools on evaluate a generic model

Once we have a model, whether it is fictitious play model or any generic model, we can use our tools to evaluate the fairness in several ways.

#### Auditing Predictions

You can audit for subgroup fairness of your predictions using the functionality within the `audit` object. These predictions can come from any arbitrary model. Auditing the predictions returns the group that failed the audit and gamma unfairness of the predictions on that group. We will be using our predictions from the previous part.

In [14]:
auditor = gerryfair.model.Auditor()
[group, gamma_unfairness] = auditor.audit(predictions, X_prime_test, y_test)

#### Plotting errors
You can also plot the errors of the model during training using the `plot_single` function in *fairness_plots.py*. Please note that these errors are returned by our fictitious play algorithm, so this is specifically for analyzing the effectiveness of our model.

In [15]:
gerryfair.fairness_plots.plot_single(errors, fp_difference, max_iters, gamma, C)

In [16]:
fair_model._fictitious_play(X_test, X_prime_test, y_test)

iteration: 1
most accurate classifier accuracy: 0.16, most acc-class unfairness: 0.018571428571428572, most acc-class size 0.26
ave error: 0.160000, gamma-unfairness: 0.018571, group_size: 0.260000, frac included ppl: 0.400000
iteration: 2
ave error: 0.160000, gamma-unfairness: 0.018571, group_size: 0.260000, frac included ppl: 0.400000
iteration: 3
ave error: 0.280000, gamma-unfairness: 0.026667, group_size: 0.280000, frac included ppl: 0.880000
iteration: 4
ave error: 0.340000, gamma-unfairness: 0.055714, group_size: 0.260000, frac included ppl: 0.940000
iteration: 5
ave error: 0.372000, gamma-unfairness: 0.062286, group_size: 0.300000, frac included ppl: 1.000000
iteration: 6
ave error: 0.373333, gamma-unfairness: 0.041667, group_size: 0.280000, frac included ppl: 1.000000
iteration: 7
ave error: 0.382857, gamma-unfairness: 0.023673, group_size: 0.220000, frac included ppl: 1.000000
iteration: 8
ave error: 0.390000, gamma-unfairness: 0.015357, group_size: 0.400000, frac included ppl

([0.16,
  0.16,
  0.28,
  0.34,
  0.37200000000000005,
  0.37333333333333335,
  0.3828571428571429,
  0.39,
  0.39555555555555555,
  0.4,
  0.3999999999999999,
  0.3966666666666667,
  0.39076923076923076,
  0.3842857142857143,
  0.376,
  0.375,
  0.3764705882352941,
  0.3844444444444444,
  0.3915789473684211,
  0.398,
  0.40285714285714286,
  0.4063636363636365,
  0.40869565217391307,
  0.40666666666666673,
  0.4023999999999999,
  0.3984615384615384,
  0.394074074074074,
  0.39,
  0.3855172413793103,
  0.382,
  0.3819354838709678,
  0.381875,
  0.3836363636363636,
  0.3847058823529412,
  0.3857142857142858,
  0.38555555555555565,
  0.3854054054054054,
  0.38526315789473686,
  0.38512820512820517,
  0.385,
  0.3843902439024391,
  0.38238095238095243,
  0.3772093023255815,
  0.3722727272727272,
  0.3675555555555556,
  0.36347826086956514,
  0.3604255319148936,
  0.3575000000000001,
  0.35510204081632657,
  0.3556,
  0.356078431372549,
  0.35692307692307684,
  0.3581132075471698,
  0.3592

: 